In [10]:
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import re

# Set up the driver
chrome_driver_path = r'D:\Downloads\chromedriver.exe'

# Set up the Selenium WebDriver using Service
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service)
object = "spam"
page_url = (f"https://shop.wegmans.com/search?search_term={object}")
driver.get(page_url)

time.sleep(5)
driver.find_element(By.XPATH, '//button[@class="li-el icon-list icon instore-context"]').click()
time.sleep(3)
driver.find_element(By.XPATH, '//button[text()="Reload"]').click()
time.sleep(3)
object_brand = driver.find_elements(By.XPATH, '//div[@class="css-131yigi"]')
time.sleep(5)
object_price = driver.find_elements(By.XPATH, '//span[@class="css-zqx11d"]')
time.sleep(5)
object_unit = driver.find_elements(By.XPATH, '//div[@class="css-1kh7mkb"]')
time.sleep(5)

object_brand
object_price

import re

def convert_kg_to_lbs(kg):
    """Convert kilograms to pounds."""
    return kg * 2.20462

def convert_oz_to_lbs(oz):
    """Convert ounces to pounds."""
    return oz * 0.0625

def convert_g_to_lbs(g):
    """Convert grams to pounds."""
    return g * 0.00220462

def extract_quantity_in_lbs(product_string):
    # Regular expression pattern to match quantity and unit (case insensitive)
    pattern = r'(\d+\.?\d*)\s*(lb|lbs|kg|kgs|oz|g)\s*'  # Handles lb, kg, oz, g

    # Find all matches in the string
    matches = re.findall(pattern, product_string, flags=re.IGNORECASE)

    # Variable to store the total quantity in pounds
    total_quantity_lbs = 0.0

    # Flag to track if we've already counted a quantity
    counted = False

    for match in matches:
        quantity, unit = match
        quantity = float(quantity)  # Convert quantity to float

        # Only count the first detected unit
        if not counted:
            if unit.lower() in ['lb', 'lbs']:  # Check for pounds
                total_quantity_lbs = quantity  # Set quantity in pounds
                counted = True  # Mark that we've counted
            elif unit.lower() in ['kg', 'kgs']:  # Check for kilograms
                total_quantity_lbs = convert_kg_to_lbs(quantity)  # Convert kg to lbs
                counted = True  # Mark that we've counted
            elif unit.lower() == 'oz':  # Check for ounces
                total_quantity_lbs = convert_oz_to_lbs(quantity)  # Convert oz to lbs
                counted = True  # Mark that we've counted
            elif unit.lower() == 'g':  # Check for grams
                total_quantity_lbs = convert_g_to_lbs(quantity)  # Convert grams to lbs
                counted = True  # Mark that we've counted

    # Return the total quantity rounded to 2 decimal places
    return round(total_quantity_lbs, 2)

# Get the number of items
number_of_items = min(len(object_brand), len(object_price))
spam_list = []

if number_of_items == 0:
    print('No items found.')
    exit()

origin = 'Wegmans'
#create a spam list with the brand, price, and quantity in lbs and also hmart as the origin
for i in range(number_of_items):
    brand = object_brand[i].text
    price = object_price[i].text
    quantity = extract_quantity_in_lbs(object_unit[i].text)
    spam_list.append([origin, brand, quantity, price])
# get rid of the discount price
for item in spam_list:
    item[3] = item[3].split('\n')[0]  # Keep only the first price
    item[3] = item[3].replace('/ea', '')  # Remove '/ea' from the price
# use pandas to create a dataframe
import pandas as pd

df = pd.DataFrame(spam_list, columns=['Origin', 'Brand', 'Quantity (lbs)', 'Price'])
print(df)

# Save the dataframe to a CSV file
df.to_csv(f'{origin}_{object}_prices.csv', index=False)


# Close the browser
driver.quit()


    Origin                                              Brand  Quantity (lbs)  \
0  Wegmans                                     SPAM , Classic            0.75   
1  Wegmans                     SPAM Meatloaf, 25% Less Sodium            0.75   
2  Wegmans                                        SPAM , Lite            0.75   
3  Wegmans                             SPAM , Single, Classic            0.16   
4  Wegmans                             Libby's Vienna Sausage            0.29   
5  Wegmans       StarKist Tuna Creations Tuna, Bold, Jalapeno            0.16   
6  Wegmans  Hormel Mary Kitchen Mary Kitchen Corned Beef H...            0.88   
7  Wegmans                                Libby's Corned Beef            0.75   
8  Wegmans        StarKist Tuna in Water, Chunk Light, 4 Pack            0.31   

    Price  
0  $4.29   
1  $4.29   
2  $4.29   
3  $2.29   
4  $1.19   
5  $1.89   
6  $3.99   
7  $6.49   
8  $4.79   
